# Kijkcijfers Project
Naam: Jens Demeyer
Studentennummer: 202398958 Vak: Machine Learning
Datum: [Datum van indienen]

## Deel 1: Scrapen van data

curl request om te testen of api werkt en om te kijken in welk formaat de json staat

In [18]:
import requests
from datetime import datetime
import json

# Stap 1: Bouw de API URL met huidige datum
vandaag = 20180127
api_url = f"https://api.cim.be/api/cim_tv_public_results_daily_views?dateDiff={vandaag}&reportType=north"

# Stap 2: Haal data op
try:
    response = requests.get(api_url)
    
    if response.status_code == 200:
        # Stap 3: Parse de JSON data
        data = response.json()
        print("data:", data)
        
    else:
        print(f"Fout bij het ophalen van de gegevens: {response.status_code}")
except requests.exceptions.RequestException as e:
    print(f"Fout bij het ophalen van de gegevens: {e}")

    response_json = json.dumps(data, indent=4)

with open('data.json', 'w') as file:
    json.dump(data, file, indent=4)

data: {'@context': '/api/contexts/CimTvPublicResultsDailyView', '@id': '/api/cim_tv_public_results_daily_views', '@type': 'hydra:Collection', 'hydra:member': [{'@id': '/api/cim_tv_public_results_daily_views/44150', '@type': 'CimTvPublicResultsDailyView', 'id': 44150, 'reportType': 'north', 'dateImport': '2018-01-29T00:00:00.000000', 'dateResult': '2018-01-27T00:00:00.000000', 'ranking': '1', 'description': 'HET 7 UUR-JOURNAAL', 'category': None, 'channel': 'EEN', 'dateDiff': '2018-01-27T00:00:00.000000', 'startTime': '19:00:04', 'rLength': '00:31:57', 'ratePerc': None, 'rateInK': '803.889', 'shr': None, 'rateInKAll': None, 'live': 0}, {'@id': '/api/cim_tv_public_results_daily_views/44151', '@type': 'CimTvPublicResultsDailyView', 'id': 44151, 'reportType': 'north', 'dateImport': '2018-01-29T00:00:00.000000', 'dateResult': '2018-01-27T00:00:00.000000', 'ranking': '2', 'description': 'FC DE KAMPIOENEN', 'category': None, 'channel': 'EEN', 'dateDiff': '2018-01-27T00:00:00.000000', 'startTi

In [25]:
import requests
import csv
from datetime import datetime

def get_viewership_data(date_yyyymmdd):
    api_url = f"https://api.cim.be/api/cim_tv_public_results_daily_views?dateDiff={date_yyyymmdd}&reportType=north"
    
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            return data.get('hydra:member', [])
        else:
            print(f"Fout bij ophalen data voor {date_yyyymmdd}: HTTP {response.status_code}")
            return []
    except requests.exceptions.RequestException as e:
        print(f"Fout bij ophalen data voor {date_yyyymmdd}: {e}")
        return []

def save_to_csv(data, filename='kijkcijfers.csv'):
    if not data:
        print("Geen data om op te slaan")
        return
    
    # Selecteer alleen de gewenste velden
    selected_fields = ['dateDiff', 'ranking', 'description', 'channel', 'startTime', 'rLength', 'rateInK']
    
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=selected_fields)
        writer.writeheader()
        
        for item in data:
            # Maak een nieuw dict met alleen de gewenste velden
            row = {field: item.get(field) for field in selected_fields}
            
            # Formatteer dateDiff naar YYYY-MM-DD
            if 'dateDiff' in row and row['dateDiff']:
                try:
                    dt = datetime.strptime(row['dateDiff'], '%Y-%m-%dT%H:%M:%S.%fZ')
                    row['dateDiff'] = dt.strftime('%Y-%m-%d')
                except ValueError:
                    pass
            
            writer.writerow(row)
    
    print(f"Data opgeslagen in {filename} ({len(data)} records)")

date_str = '20250518'  # Format: YYYYMMDD
data = get_viewership_data(date_str)

if data:
    save_to_csv(data)
else:
    print("Geen data gevonden voor deze datum")

Data opgeslagen in kijkcijfers.csv (20 records)


In [6]:
import requests
import csv
from datetime import datetime, timedelta
import time
from tqdm import tqdm
import os

def get_viewership_data(date_yyyymmdd, max_retries=3):
    """Haal kijkcijferdata op voor specifieke datum"""
    api_url = f"https://api.cim.be/api/cim_tv_public_results_daily_views?dateDiff={date_yyyymmdd}&reportType=north"
    
    for attempt in range(max_retries):
        try:
            response = requests.get(api_url, timeout=30)
            if response.status_code == 200:
                return response.json().get('hydra:member', [])
            return []
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"\nFout voor {date_yyyymmdd}: {str(e)[:100]}...")
            time.sleep(2 ** attempt)
    return []

def append_to_csv(data, filename='kijkcijfers.csv'):
    """Voeg data toe aan CSV zonder duplicate checking"""
    if not data:
        return
    
    fields = ['dateDiff', 'ranking', 'description', 'channel', 'startTime', 'rLength', 'rateInK']
    file_exists = os.path.isfile(filename)
    
    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fields)
        if not file_exists:
            writer.writeheader()
        
        for item in data:
            row = {field: item.get(field) for field in fields}
            if 'dateDiff' in row and row['dateDiff']:
                try:
                    dt = datetime.strptime(row['dateDiff'], '%Y-%m-%dT%H:%M:%S.%fZ')
                    row['dateDiff'] = dt.strftime('%Y-%m-%d')
                except ValueError:
                    pass
            writer.writerow(row)

def scrape_with_progress(start_date, end_date):
    """Scrape data voor een periode met voortgangsbalk"""
    date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]
    
    with tqdm(total=len(date_range), desc="Data scrapen", unit="dag") as pbar:
        for date in date_range:
            date_str = date.strftime('%Y%m%d')
            data = get_viewership_data(date_str)
            append_to_csv(data)
            pbar.update(1)
            time.sleep(0.5)  # Mild rate limiting

    print(f"\nData succesvol toegevoegd aan kijkcijfers.csv")

# Voorbeeldgebruik:
if __name__ == "__main__":
    scrape_with_progress(
        start_date=datetime(2019, 11, 1),
        end_date=datetime(2025, 5, 20)
    )

Data scrapen: 100%|██████████| 2028/2028 [29:59<00:00,  1.13dag/s]  


Data succesvol toegevoegd aan kijkcijfers.csv


In [11]:
import csv
from datetime import datetime

def clean_holidays_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile, \
         open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        
        reader = csv.DictReader(infile, delimiter=';')
        writer = csv.DictWriter(outfile, fieldnames=['date', 'holiday_name'], delimiter=';')
        writer.writeheader()
        
        for row in reader:
            # Pak alleen de NL naam (alles tussen "NL " en de volgende komma)
            name_nl = row['Name'].split('NL ')[1].split(',')[0].strip()
            
            # Schrijf alleen de gewenste velden
            writer.writerow({
                'date': row['StartDate'],
                'holiday_name': name_nl
            })

# Pas aan voor jouw bestandsnamen
clean_holidays_file('holidays.public.csv', 'holidays_clean.csv')

In [12]:
import csv

def clean_school_holidays(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile, \
         open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        
        reader = csv.DictReader(infile, delimiter=';')
        writer = csv.DictWriter(outfile, fieldnames=['start_date', 'end_date', 'holiday_name'], delimiter=';')
        writer.writeheader()
        
        for row in reader:
            # Pak alleen de NL naam (alles tussen "NL " en de volgende komma)
            name_nl = row['Name'].split('NL ')[1].split(',')[0].strip()
            
            # Schrijf alleen de gewenste velden
            writer.writerow({
                'start_date': row['StartDate'],
                'end_date': row['EndDate'],
                'holiday_name': name_nl
            })

# Pas aan voor jouw bestandsnamen
clean_school_holidays('holidays.school.vlaanderen.csv', 'school_holidays_clean.csv')

Later heb ik manueel de data voor 2016-2019 toegevoegd

In [27]:
import pandas as pd
from datetime import datetime

# Laad de hoofddata
df_main = pd.read_csv('./Data/CleanData/kijkcijfers.csv', delimiter=',')

# Laad de feestdagen data
df_holidays = pd.read_csv('./Data/CleanData/holidays_clean.csv', delimiter=';')
df_holidays['date'] = pd.to_datetime(df_holidays['date'])

# Laad de schoolvakanties data
df_school_holidays = pd.read_csv('./Data/CleanData/school_holidays_clean.csv', delimiter=';')
df_school_holidays['start_date'] = pd.to_datetime(df_school_holidays['start_date'])
df_school_holidays['end_date'] = pd.to_datetime(df_school_holidays['end_date'])

# Converteer dateDiff naar datetime
df_main['dateDiff'] = pd.to_datetime(df_main['dateDiff'])

# Maak is_holiday feature
def check_holiday(date):
    # Check publieke feestdagen
    if date in df_holidays['date'].values:
        return 1
    
    # Check schoolvakanties
    for _, row in df_school_holidays.iterrows():
        if row['start_date'] <= date <= row['end_date']:
            return 1
    
    return 0

df_main['is_holiday'] = df_main['dateDiff'].apply(check_holiday)

print(df_main.head())

print(df_main[df_main['dateDiff'] == pd.to_datetime('2016-11-01')].head())


    dateDiff  ranking         description channel startTime   rLength  \
0 2016-10-01        4    IEDEREEN BEROEMD     EEN  19:38:10  00:29:01   
1 2016-10-01        5      COMEDY TOPPERS     VTM  19:52:06  00:24:40   
2 2016-10-01        6        THE FUGITIVE     EEN  21:22:25  02:01:25   
3 2016-10-01        7      NIEUWS 19U VTM     VTM  18:59:49  00:42:27   
4 2016-10-01        8  HET 1 UUR-JOURNAAL     EEN  13:00:05  00:22:48   

   rateInK  is_holiday  
0  523.610           0  
1  496.216           0  
2  447.427           0  
3  424.041           0  
4  369.066           0  
      dateDiff  ranking                 description channel startTime  \
601 2016-11-01        1                       THUIS     EEN  20:15:47   
602 2016-11-01        2          HET 7 UUR-JOURNAAL     EEN  19:00:04   
603 2016-11-01        3            IEDEREEN BEROEMD     EEN  19:46:21   
604 2016-11-01        4  DE SLIMSTE MENS TER WERELD    VIER  21:34:44   
605 2016-11-01        5                     FA

In [31]:
import pandas as pd
from datetime import datetime

df_main['weekday'] = df_main['dateDiff'].dt.dayofweek
df_main['month'] = df_main['dateDiff'].dt.month

# 3. Voeg is_primetime toe (18:30-22:00)
# Eerst startTime omzetten naar datetime voor vergelijking
def fix_invalid_time(t):
    # Corrigeer "24:xx:xx" naar "00:xx:xx"
    if isinstance(t, str) and t.startswith("24:"):
        return "00:" + t[3:]
    return t

df_main['startTime_fixed'] = df_main['startTime'].apply(fix_invalid_time)
df_main['startTime'] = pd.to_datetime(df_main['startTime_fixed'], format='%H:%M:%S', errors='coerce').dt.time

# Maak primetime begin en eind
primetime_start = pd.to_datetime('18:30:00').time()
primetime_end = pd.to_datetime('22:00:00').time()

df_main['is_primetime'] = df_main['startTime'].apply(
    lambda x: 1 if pd.notnull(x) and primetime_start <= x <= primetime_end else 0
)

df_main['is_weekend'] = df_main['weekday'].apply(lambda x: 1 if x >= 5 else 0)

# Optioneel: Toon resultaat
print(df_main[['dateDiff', 'startTime', 'weekday', 'month', 'is_primetime', 'is_weekend']].head(10))

print(df_main.head())


    dateDiff startTime  weekday  month  is_primetime  is_weekend
0 2016-10-01  19:38:10        5     10             1           1
1 2016-10-01  19:52:06        5     10             1           1
2 2016-10-01  21:22:25        5     10             1           1
3 2016-10-01  18:59:49        5     10             1           1
4 2016-10-01  13:00:05        5     10             0           1
5 2016-10-01  14:29:41        5     10             0           1
6 2016-10-01  22:57:22        5     10             0           1
7 2016-10-01  20:32:14        5     10             1           1
8 2016-10-01  18:12:41        5     10             0           1
9 2016-10-01  23:24:38        5     10             0           1
    dateDiff  ranking         description channel startTime   rLength  \
0 2016-10-01        4    IEDEREEN BEROEMD     EEN  19:38:10  00:29:01   
1 2016-10-01        5      COMEDY TOPPERS     VTM  19:52:06  00:24:40   
2 2016-10-01        6        THE FUGITIVE     EEN  21:22:25  02:01

In [37]:
import pandas as pd
import re

# Lijst van nieuwsgerelateerde trefwoorden (kan uitgebreid worden)
news_keywords = [
    'journaal', 'nieuws', 'sportweekend'
]

# Lijst van sportgerelateerde trefwoorden (kan uitgebreid worden)
sports_keywords = [
    'voetbal', 'wielrennen', 'veldrijden','golf', 'boxen','snooker', 'tennis','atletiek', 'zwemmen',
    'volleybal', 'basketbal', 'handbal', 'hockey', 'formule 1', 'f1', 'vb', 'o.s.', 'darts'
]

# Maak regex patronen voor efficiënte matching
news_pattern = re.compile('|'.join(news_keywords), flags=re.IGNORECASE)
sports_pattern = re.compile('|'.join(sports_keywords), flags=re.IGNORECASE)

# Voeg features toe
df_main['is_news'] = df_main['description'].apply(
    lambda x: 1 if news_pattern.search(str(x)) else 0
)

df_main['is_sports'] = df_main['description'].apply(
    lambda x: 1 if sports_pattern.search(str(x)) else 0
)

# Toon resultaten
print("\nAantal nieuwsprogramma's:", df_main['is_news'].sum())
print("Aantal sportprogramma's:", df_main['is_sports'].sum())
print("\nVoorbeeld van geclassificeerde programma's:")
print(df_main[['description', 'is_news', 'is_sports']].sample(10))


Aantal nieuwsprogramma's: 15498
Aantal sportprogramma's: 4038

Voorbeeld van geclassificeerde programma's:
                            description  is_news  is_sports
45997                      NCIS, HAWAII        0          0
24426                             THUIS        0          0
6646                              THUIS        0          0
5347                            FAMILIE        0          0
609                     HET GOEIE LEVEN        0          0
3850                   VB. C1 UEFA 1/4F        0          1
16222  THE VOICE SENIOR - THE KNOCKOUTS        0          0
28039                   DE BUURTPOLITIE        0          0
56595               SAVING PRIVATE RYAN        0          0
13662                    THE KARATE KID        0          0


In [59]:
# Converteer eerst rLength naar minuten als je dit nog niet gedaan hebt
df_main['duration_min'] = round(pd.to_timedelta(df_main['rLength']).dt.total_seconds() / 60, 2)

# Maak de is_film feature (1 als duration_min > 105, anders 0)
df_main['is_film'] = (df_main['duration_min'] > 95).astype(int)

df_main.loc[df_main['is_sports'] == 1, 'is_film'] = 0

# Voorbeelden tonen
print("\nVoorbeeld van film-classificatie:")
print(df_main[['description', 'rLength', 'duration_min', 'is_film']].sample(10))


Voorbeeld van film-classificatie:
                                     description   rLength  duration_min  \
2850                     BUURMAN, WAT DOET U NU?  00:29:29         29.48   
36533                             NIEUWS 19U VTM  00:53:39         53.65   
60209                            DAGELIJKSE KOST  00:15:31         15.52   
26725                                     POIROT  01:28:32         88.53   
62999                         HET 7 UUR-JOURNAAL  00:39:05         39.08   
40770  VB. C1 UEFA 1/2F - MANCHESTER C./R.MADRID  01:36:48         96.80   
54776                  HET ECHTE LEVEN IN DE ZOO  00:26:12         26.20   
26529                   THE BROKENWOOD MYSTERIES  01:26:13         86.22   
59642                           IEDEREEN BEROEMD  00:23:27         23.45   
45863                             NIEUWS 19U VTM  00:50:50         50.83   

       is_film  
2850         0  
36533        0  
60209        0  
26725        0  
62999        0  
40770        0  
54776    

In [60]:
print(df_main.head())

    dateDiff  ranking         description channel startTime   rLength  \
0 2016-10-01        4    IEDEREEN BEROEMD     EEN  19:38:10  00:29:01   
1 2016-10-01        5      COMEDY TOPPERS     VTM  19:52:06  00:24:40   
2 2016-10-01        6        THE FUGITIVE     EEN  21:22:25  02:01:25   
3 2016-10-01        7      NIEUWS 19U VTM     VTM  18:59:49  00:42:27   
4 2016-10-01        8  HET 1 UUR-JOURNAAL     EEN  13:00:05  00:22:48   

   rateInK  is_holiday  weekday  month startTime_fixed  is_primetime  \
0  523.610           0        5     10        19:38:10             1   
1  496.216           0        5     10        19:52:06             1   
2  447.427           0        5     10        21:22:25             1   
3  424.041           0        5     10        18:59:49             1   
4  369.066           0        5     10        13:00:05             0   

   is_weekend  is_news  is_sports  duration_min  is_film  
0           1        0          0         29.02        0  
1         

In [70]:
#rateInK veranderen van naam naar aantal_kijkers
df_main.rename(columns={'rateInK': 'aantal_kijkers'}, inplace=True)

#zet de rateInK kolom om naar numeriek
df_main['rateInK'] = pd.to_numeric(df_main['aantal_kijkers'], errors='coerce')

# Definieer de gewenste kolommen in de juiste volgorde
desired_columns = [
    'description', 
    'channel', 
    'startTime_fixed', 
    'duration_min', 
    'weekday', 
    'month', 
    'aantal_kijkers', 
    'is_primetime', 
    'is_weekend', 
    'is_holiday',
    'is_news', 
    'is_sports', 
    'is_film'
]

# Filter de DataFrame om alleen deze kolommen te behouden
df_final = df_main[desired_columns]

# Toon het resultaat
print(df_final.sample(10))

                        description channel startTime_fixed  duration_min  \
19750                NIEUWS 19U VTM     VTM        18:59:54         52.37   
30083  VELE HEMELS BOVEN DE ZEVENDE     EEN        20:44:30        112.10   
10023             VB. C3 UEFA 1/16F  CANVAS        21:06:32         92.93   
11692                      AFSCHEID     EEN        21:33:39         37.97   
9490               IEDEREEN BEROEMD     EEN        19:37:52         29.53   
19074             HET JOURNAAL LAAT     EEN        23:13:57         16.82   
50417                         THUIS   VRT 1        13:33:47         23.42   
28718              DIEREN IN NESTEN     EEN        17:44:29         18.42   
27499                      TER ZAKE  Canvas        20:00:05         35.38   
43136      ZWEMMEN. EK.MUNCHEN (S.)     EEN        23:13:27         20.33   

       weekday  month  aantal_kijkers  is_primetime  is_weekend  is_holiday  \
19750        0      6         534.392             1           0          

In [71]:
df_final.to_csv('./Data/CleanData/kijkcijfers_final_features.csv', index=False, sep=';')

In [73]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Basisstatistieken
print("⏳ Basisstatistieken:")
print(df_final.describe())

# Categorische variabelen analyse
print("\n📊 Channel distributie:")
print(df_final['channel'].value_counts(normalize=True) * 100)

# Correlatiematrix
print("\n🔗 Correlatiematrix:")
print(df_final.corr(numeric_only=True))

# Tijdsanalyse
print("\n📅 Maandelijkse trends:")
print(df_final.groupby('month')['aantal_kijkers'].mean().sort_values(ascending=False))

⏳ Basisstatistieken:
       duration_min       weekday         month  aantal_kijkers  is_primetime  \
count  63365.000000  63398.000000  63398.000000    59804.000000  63398.000000   
mean      44.707601      3.002177      6.478580      633.625228      0.582258   
std       29.281509      2.001883      3.502318    10701.902084      0.493191   
min       15.000000      0.000000      1.000000       40.984000      0.000000   
25%       26.120000      1.000000      3.000000      224.150750      0.000000   
50%       39.370000      3.000000      6.000000      339.547000      1.000000   
75%       51.070000      5.000000     10.000000      556.486500      1.000000   
max      589.320000      6.000000     12.000000   957556.000000      1.000000   

         is_weekend    is_holiday       is_news     is_sports       is_film  
count  63398.000000  63398.000000  63398.000000  63398.000000  63398.000000  
mean       0.286586      0.296113      0.244456      0.063693      0.036925  
std        0.45